In [ ]:
#import of pre-requsite packages and function
from datetime import datetime, timedelta
from airflow import DAG
from airflow.operators.dummy_operator import DummyOperator
from airflow.operators.python_operator import PythonOperator
from airflow.providers.amazon.aws.operators.glue import AwsGlueJobOperator


#email adress to alert
ALERT_EMAIL_ADDRESSES=["on_call_team@zocket.com","business_dl@zocket.com"]

# Custom Python functions for pre API checks
def extract_facebook_ads_readiness():
    """
    logic   to check status of api from facebook endpoints like active ,4xx or 5xx states

    """
    pass

def extract_google_ads_readiness():
    """
    logic   to check status of api from google endpoints like active ,4xx or 5xx states

    """
    pass


# Default arguments for the DAG
default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'email': ALERT_EMAIL_ADDRESSES,
    'email_on_failure': True,
    'email_on_retry': True,
    'max_active_runs': 1,
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}

# Define the DAG object
with DAG(
    'etl_pipeline',
    default_args=default_args,
    description='ETL pipeline for Facebook and Google Ads data',
    schedule_interval=timedelta(days=1),
    start_date=datetime(2023, 1, 1),
    catchup=False,
) as dag:

    start = DummyOperator(
        task_id='start'
    )



    readiness_check_extract_google_ads = PythonOperator(
        task_id='extract_google_ads',
        python_callable=extract_google_ads_readiness
    )


    transform_in_glue_google_ads = AwsGlueJobOperator(
        task_id='transform_in_glue_google_ads',
        job_name='load_data_to_S3_raw_google_ads',
        script_location='s3://zocket-ad-bronze/scripts/load_to_stg_apidata_script_google.py',
        iam_role_name='your_glue_service_role'
    )

    readiness_check_extract_facebook_ads = PythonOperator(
        task_id='extract_facebook_ads',
        python_callable=extract_facebook_ads_readiness
    )
    transform_in_glue_facebook_ads = AwsGlueJobOperator(
        task_id='transform_in_glue_facebook_ads',
        job_name='load_data_to_S3_raw_facebook_ads',
        script_location='s3://zocket-ad-bronze/scripts/load_to_stg_apidata_script_fb.py',
        iam_role_name='glue_role_as_per_arg'
    )


    end = DummyOperator(
        task_id='end'
    )

    # Define the task dependencies
    start >> [readiness_check_extract_google_ads >>transform_in_glue_google_ads , readiness_check_extract_facebook_ads >>transform_in_glue_facebook_ads] >>end
